In [72]:
import os
import json
import pandas as pd
import traceback

In [73]:
from langchain.chat_models import ChatOpenAI


In [74]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [ ]:
KEY=os.getenv("KEY")

In [76]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [78]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [79]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [80]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [81]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [82]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [83]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [84]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [85]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [86]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [87]:
file_path=r"F:\openai\project\data.txt"

In [88]:
file_path

'F:\\openai\\project\\data.txt'

In [89]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [90]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [91]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [92]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"


In [93]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{
"1": {
"mcq": "Who coined the term machine learning in 1959?",
"options": {
"a": "Donald Hebb",
"b": "Raytheon Company",
"c": "Arthur Samuel",
"d": "Tom M. Mitchell"
},
"correct": "c"
},
"2": {
"mcq": "What was the purpose of Cybertron, developed by Raytheon Company in the early 1960s?",
"options": {
"a": "To analyze weather patterns",
"b": "To recognize patterns in sonar signals, electrocardiograms, and speech patterns",
"c": "To play chess",
"d": "To predict stock market tre

In [94]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1814
Prompt Tokens:1383
Completion Tokens:431
Total Cost:0.0029365


In [95]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [96]:
quiz=response.get("quiz")

In [97]:
quiz=json.loads(quiz)

In [98]:
quiz

{'1': {'mcq': 'Who coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Raytheon Company',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '2': {'mcq': 'What was the purpose of Cybertron, developed by Raytheon Company in the early 1960s?',
  'options': {'a': 'To analyze weather patterns',
   'b': 'To recognize patterns in sonar signals, electrocardiograms, and speech patterns',
   'c': 'To play chess',
   'd': 'To predict stock market trends'},
  'correct': 'b'},
 '3': {'mcq': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'options': {'a': 'Improving performance at tasks in T, as measured by P, with experience E',
   'b': 'Analyzing human cognitive processes',
   'c': 'Recognizing patterns in data',
   'd': 'Classifying data based on models'},
  'correct': 'a'},
 '4': {'mcq': 'What is one of the objectives of modern-day machine learning?',
  'options': {'a': 'To predict the weather acc

In [99]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [100]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Donald Hebb | b: Raytheon Company | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What was the purpose of Cybertron, developed by Raytheon Company in the early 1960s?',
  'Choices': 'a: To analyze weather patterns | b: To recognize patterns in sonar signals, electrocardiograms, and speech patterns | c: To play chess | d: To predict stock market trends',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of machine learning algorithms?',
  'Choices': 'a: Improving performance at tasks in T, as measured by P, with experience E | b: Analyzing human cognitive processes | c: Recognizing patterns in data | d: Classifying data based on models',
  'Correct': 'a'},
 {'MCQ': 'What is one of the objectives of modern-day machine learning?',
  'Choices': 'a: To predict the weather accurately | b: To make predictions for future outcomes based on models | c: To study human 

In [101]:
quiz=pd.DataFrame(quiz_table_data)

In [102]:
# quiz.to_csv("machinelearning.csv",index=False)

In [103]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'11_15_2024_11_48_55'